In [2]:
import logging
import os
from fetch_data import (
    fetch_finland_cases_age_weekly, fetch_thl_cases_erva_weekly,
    static_population_erva_age
)
import pandas as pd
import numpy as np

In [3]:
curr_dir = os.getcwd()
stats_dir = os.path.join(curr_dir, 'stats')
logger = logging.getLogger(__name__)
curr_dir

'/Users/aponcedeleonch/Documents/Aalto/Courses/MasterThesis/Code/jeta'

In [3]:
age_file = os.path.join(stats_dir, 'erva_population_age_2020.csv')
pop_age, pop_age_prop = static_population_erva_age(logger, age_file)
pop_age

,erva,Age,Total
1,All,0-14,860861
2,All,15-64,3416994
3,All,65+,1255938
5,HYKS,0-14,344271
6,HYKS,15-64,1411278
7,HYKS,65+,442633
9,TYKS,0-14,130240
10,TYKS,15-64,524587
11,TYKS,65+,214177
13,TAYS,0-14,139549


In [4]:
erva_cases = fetch_thl_cases_erva_weekly(logger)
erva_cases

,Time,erva,val
0,All times,All,81261.0
1,All times,HYKS,52407.0
2,All times,KYS,5381.0
3,All times,OYS,4469.0
4,All times,TAYS,6403.0
...,...,...,...
737,Year 2021 Week 52,KYS,0.0
738,Year 2021 Week 52,OYS,0.0
739,Year 2021 Week 52,TAYS,0.0
740,Year 2021 Week 52,TYKS,0.0


In [40]:
test_erva = erva_cases.loc[erva_cases['Time'] > 'Year 2020 Week 10', ]
test_erva = test_erva.loc[test_erva['Time'] < 'Year 2021 Week 15', ]
test_erva = test_erva.loc[test_erva['erva'] == 'All', ]
np.mean(test_erva.val.values)

1424.859649122807

In [48]:
cases_age, cases_age_prop = fetch_finland_cases_age_weekly(logger)
cases_age

,Time,age_group,val
0,All times,0-14,6559
1,All times,15-64,70327
2,All times,65+,4375
3,All times,All,81261
4,Year 2020 Week 01,0-14,0
...,...,...,...
419,Year 2021 Week 51,All,0
420,Year 2021 Week 52,0-14,0
421,Year 2021 Week 52,15-64,0
422,Year 2021 Week 52,65+,0


In [6]:
pop_erva = pop_age.groupby(['erva']).sum()
pop_erva_all_ages = pop_erva.to_dict()
pop_erva_all_ages = pop_erva_all_ages['Total']
pop_erva_all_ages

{'All': 5533793,
 'HYKS': 2198182,
 'KYS': 797234,
 'OYS': 736563,
 'TAYS': 902681,
 'TYKS': 869004,
 'Åland': 30129}

In [7]:
age_groups = pd.unique(pop_age_prop['Age'])
ervas = pd.unique(pop_age_prop['erva'])
pop_age_dict = {}
for row in pop_age.values.tolist():
    erva, age, age_prop = row
    if erva not in pop_age_dict:
        pop_age_dict[erva] = {}
    pop_age_dict[erva][age] = age_prop
print(pop_age_dict)

{'All': {'0-14': 860861, '15-64': 3416994, '65+': 1255938}, 'HYKS': {'0-14': 344271, '15-64': 1411278, '65+': 442633}, 'TYKS': {'0-14': 130240, '15-64': 524587, '65+': 214177}, 'TAYS': {'0-14': 139549, '15-64': 548166, '65+': 214966}, 'KYS': {'0-14': 114129, '15-64': 473988, '65+': 209117}, 'OYS': {'0-14': 127698, '15-64': 440751, '65+': 168114}, 'Åland': {'0-14': 4974, '15-64': 18224, '65+': 6931}}


In [8]:
list_erva_cases = erva_cases.values.tolist()
cases_by_age_erva = []
for row in list_erva_cases:
    time, erva, cases = row
    for age in age_groups:
        if age == 'All' or erva == 'All':
            continue
        age_cases_erva = np.round(cases*pop_age_dict[erva][age]/pop_erva_all_ages[erva])
        new_line = [erva, time, age, age_cases_erva]
        cases_by_age_erva.append(new_line)
columns = ['erva', 'Time', 'Age', 'val']
cases_by_age_erva = pd.DataFrame(data=cases_by_age_erva, columns=columns)
cases_by_age_erva

,erva,Time,Age,val
0,HYKS,All times,0-14,8208.0
1,HYKS,All times,15-64,33646.0
2,HYKS,All times,65+,10553.0
3,KYS,All times,0-14,770.0
4,KYS,All times,15-64,3199.0
...,...,...,...,...
1903,TYKS,Year 2021 Week 52,15-64,0.0
1904,TYKS,Year 2021 Week 52,65+,0.0
1905,Åland,Year 2021 Week 52,0-14,0.0
1906,Åland,Year 2021 Week 52,15-64,0.0


In [9]:
age_1 = cases_by_age_erva.groupby(['Time', 'Age'], as_index=False).sum()
age_1.loc[age_1['Time'] == 'Year 2021 Week 10', ]
# vals_age_erva = cases_by_age_erva.loc[cases_by_age_erva['Time'] == 'Year 2021 Week 10', 'val'].values

,Time,Age,val
189,Year 2021 Week 10,0-14,767.0
190,Year 2021 Week 10,15-64,3110.0
191,Year 2021 Week 10,65+,1058.0


In [10]:
cases_age.loc[(cases_age['Time'] == 'Year 2021 Week 10') & (cases_age['age_group'] != 'All'), ]

,Time,age_group,val
252,Year 2021 Week 10,0-14,549
253,Year 2021 Week 10,15-64,4201
254,Year 2021 Week 10,65+,186


In [11]:
def compare_age(cases_age, compare_df):
    compare_df = compare_df.groupby(['Time', 'Age'], as_index=False).sum()
    times = pd.unique(cases_age['Time'])
    distance = 0
    final_times = 0
    for time in times:
        if time == 'All times':
            continue
        if time <= 'Year 2020 Week 10':
            continue
        if time >= 'Year 2021 Week 15':
            continue
        final_times += 1
        orig_age_cases = cases_age.loc[(cases_age['Time'] == time) & (cases_age['age_group'] != 'All'), ]
        orig_age_cases = orig_age_cases.sort_values('age_group')
        compare_age_cases = compare_df.loc[compare_df['Time'] == time, ]
        compare_age_cases = compare_age_cases.sort_values('Age')
        distance += np.sqrt(np.sum((orig_age_cases.val.values - compare_age_cases.val.values)**2))
    avg_distance = distance / final_times
    return avg_distance

In [12]:
compare_age(cases_age, cases_by_age_erva)

429.57427564350206

In [13]:
erva_1 = cases_by_age_erva.groupby(['Time', 'erva'], as_index=False).sum()
erva_1.loc[erva_1['Time'] == 'Year 2021 Week 10', ]

,Time,erva,val
378,Year 2021 Week 10,HYKS,3421.0
379,Year 2021 Week 10,KYS,203.0
380,Year 2021 Week 10,OYS,128.0
381,Year 2021 Week 10,TAYS,459.0
382,Year 2021 Week 10,TYKS,706.0
383,Year 2021 Week 10,Åland,18.0


In [14]:
erva_cases.loc[erva_cases['Time'] == 'Year 2021 Week 10', ]

,Time,erva,val
441,Year 2021 Week 10,All,4936.0
442,Year 2021 Week 10,HYKS,3421.0
443,Year 2021 Week 10,KYS,203.0
444,Year 2021 Week 10,OYS,129.0
445,Year 2021 Week 10,TAYS,459.0
446,Year 2021 Week 10,TYKS,705.0
447,Year 2021 Week 10,Åland,19.0


In [15]:
def compare_ervas(erva_cases, compare_df):
    compare_df = compare_df.groupby(['Time', 'erva'], as_index=False).sum()
    times = pd.unique(erva_cases['Time'])
    distance = 0
    final_times = 0
    for time in times:
        if time == 'All times':
            continue
        if time <= 'Year 2020 Week 10':
            continue
        if time >= 'Year 2021 Week 15':
            continue
        final_times += 1
        orig_erva_cases = erva_cases.loc[(erva_cases['Time'] == time) & (erva_cases['erva'] != 'All'), ]
        orig_erva_cases = orig_erva_cases.sort_values('erva')
        compare_erva_cases = compare_df.loc[compare_df['Time'] == time, ]
        compare_erva_cases = compare_erva_cases.sort_values('erva')
        distance += np.sqrt(np.sum((orig_erva_cases.val.values - compare_erva_cases.val.values)**2))
    avg_distance = distance / final_times
    return avg_distance

In [16]:
compare_ervas(erva_cases, cases_by_age_erva)

1.125275047508998

In [17]:
list_cases_age = cases_age.values.tolist()
cases_by_age_erva_2 = []
for row in list_cases_age:
    time, age, cases = row
    for erva in ervas:
        if age == 'All' or erva == 'All':
            continue
        age_cases_erva = np.round(cases*pop_age_dict[erva][age]/pop_age_dict['All'][age])
        new_line = [erva, time, age, age_cases_erva]
        cases_by_age_erva_2.append(new_line)
columns = ['erva', 'Time', 'Age', 'val']
cases_by_age_erva_2 = pd.DataFrame(data=cases_by_age_erva_2, columns=columns)
cases_by_age_erva_2

,erva,Time,Age,val
0,HYKS,All times,0-14,2623.0
1,TYKS,All times,0-14,992.0
2,TAYS,All times,0-14,1063.0
3,KYS,All times,0-14,870.0
4,OYS,All times,0-14,973.0
...,...,...,...,...
1903,TYKS,Year 2021 Week 52,65+,0.0
1904,TAYS,Year 2021 Week 52,65+,0.0
1905,KYS,Year 2021 Week 52,65+,0.0
1906,OYS,Year 2021 Week 52,65+,0.0


In [18]:
cases_by_age_erva_2.loc[cases_by_age_erva_2['Time'] == 'Year 2021 Week 10', ]

,erva,Time,Age,val
1134,HYKS,Year 2021 Week 10,0-14,220.0
1135,TYKS,Year 2021 Week 10,0-14,83.0
1136,TAYS,Year 2021 Week 10,0-14,89.0
1137,KYS,Year 2021 Week 10,0-14,73.0
1138,OYS,Year 2021 Week 10,0-14,81.0
1139,Åland,Year 2021 Week 10,0-14,3.0
1140,HYKS,Year 2021 Week 10,15-64,1735.0
1141,TYKS,Year 2021 Week 10,15-64,645.0
1142,TAYS,Year 2021 Week 10,15-64,674.0
1143,KYS,Year 2021 Week 10,15-64,583.0


In [37]:
# age_2 = cases_by_age_erva_2.groupby(['Time', 'Age'], as_index=False).sum()
# age_2.loc[age_2['Time'] == 'Year 2021 Week 10', ]
# cases_age.loc[cases_age['Time'] == 'Year 2021 Week 10', ]
# compare_age(cases_age, cases_by_age_erva_2)

In [22]:
erva_2 = cases_by_age_erva_2.groupby(['Time', 'erva'], as_index=False).sum()
erva_2.loc[erva_2['Time'] == 'Year 2021 Week 10', ]

,Time,erva,val
378,Year 2021 Week 10,HYKS,2021.0
379,Year 2021 Week 10,KYS,687.0
380,Year 2021 Week 10,OYS,648.0
381,Year 2021 Week 10,TAYS,795.0
382,Year 2021 Week 10,TYKS,760.0
383,Year 2021 Week 10,Åland,26.0


In [23]:
erva_cases.loc[erva_cases['Time'] == 'Year 2021 Week 10', ]

,Time,erva,val
441,Year 2021 Week 10,All,4936.0
442,Year 2021 Week 10,HYKS,3421.0
443,Year 2021 Week 10,KYS,203.0
444,Year 2021 Week 10,OYS,129.0
445,Year 2021 Week 10,TAYS,459.0
446,Year 2021 Week 10,TYKS,705.0
447,Year 2021 Week 10,Åland,19.0


In [24]:
compare_ervas(erva_cases, cases_by_age_erva_2)

400.04991515888537

In [25]:
65.552390 + 31.718860 + 31.835708 + 30.969492 + 24.897092 + 1.026457

185.999999

In [26]:
pop_age_dict['All']

{'0-14': 860861, '15-64': 3416994, '65+': 1255938}

In [35]:
np.ceil(0.4)

1.0

In [41]:
from scipy.stats import multinomial

multinomial.rvs(30, [0.5, 0.25, 0.25])

array([18,  6,  6])

In [49]:
cases_age_prop.loc[(cases_age_prop['Time'] == 'Year 2021 Week 10') & (cases_age_prop['age_group'] != 'All'), ]

,Time,age_group,val
252,Year 2021 Week 10,0-14,0.111224
253,Year 2021 Week 10,15-64,0.851094
254,Year 2021 Week 10,65+,0.037682


In [54]:
ages = pd.unique(cases_age_prop.sort_values('age_group')['age_group'])
list_erva_cases = erva_cases.values.tolist()
cases_by_age_erva = []
for row in list_erva_cases:
    time, erva, cases = row
    if erva == 'All':
        continue
    probs = cases_age_prop.loc[(cases_age_prop['Time'] == time) & (cases_age_prop['age_group'] != 'All'), ]
    probs = probs.sort_values('age_group')
    probs = probs.val.values
    sampled_peopl_ages = multinomial.rvs(cases, probs)
    i = 0
    for age in ages:
        if age == 'All':
            continue
        new_line = [erva, time, age, sampled_peopl_ages[i]]
        i += 1
        cases_by_age_erva.append(new_line)
columns = ['erva', 'Time', 'Age', 'val']
cases_by_age_erva = pd.DataFrame(data=cases_by_age_erva, columns=columns)
cases_by_age_erva

,erva,Time,Age,val
0,HYKS,All times,0-14,4315
1,HYKS,All times,15-64,45229
2,HYKS,All times,65+,2863
3,KYS,All times,0-14,404
4,KYS,All times,15-64,4683
...,...,...,...,...
1903,TYKS,Year 2021 Week 52,15-64,0
1904,TYKS,Year 2021 Week 52,65+,0
1905,Åland,Year 2021 Week 52,0-14,0
1906,Åland,Year 2021 Week 52,15-64,0


In [55]:
compare_ervas(erva_cases, cases_by_age_erva)

0.0

In [56]:
compare_age(cases_age, cases_by_age_erva)

13.555577846928456

In [59]:
cases_by_age_erva.loc[cases_by_age_erva['Time'] == 'Year 2021 Week 10', ]

,erva,Time,Age,val
1134,HYKS,Year 2021 Week 10,0-14,332
1135,HYKS,Year 2021 Week 10,15-64,2957
1136,HYKS,Year 2021 Week 10,65+,132
1137,KYS,Year 2021 Week 10,0-14,17
1138,KYS,Year 2021 Week 10,15-64,179
1139,KYS,Year 2021 Week 10,65+,7
1140,OYS,Year 2021 Week 10,0-14,15
1141,OYS,Year 2021 Week 10,15-64,106
1142,OYS,Year 2021 Week 10,65+,8
1143,TAYS,Year 2021 Week 10,0-14,47


In [62]:
erva_agg = cases_by_age_erva.groupby(['Time', 'erva'], as_index=False).sum()
erva_agg.loc[erva_agg['Time'] == 'Year 2021 Week 10', ]

,Time,erva,val
378,Year 2021 Week 10,HYKS,3421
379,Year 2021 Week 10,KYS,203
380,Year 2021 Week 10,OYS,129
381,Year 2021 Week 10,TAYS,459
382,Year 2021 Week 10,TYKS,705
383,Year 2021 Week 10,Åland,19


In [60]:
erva_cases.loc[erva_cases['Time'] == 'Year 2021 Week 10', ]

,Time,erva,val
441,Year 2021 Week 10,All,4936.0
442,Year 2021 Week 10,HYKS,3421.0
443,Year 2021 Week 10,KYS,203.0
444,Year 2021 Week 10,OYS,129.0
445,Year 2021 Week 10,TAYS,459.0
446,Year 2021 Week 10,TYKS,705.0
447,Year 2021 Week 10,Åland,19.0


In [63]:
age_agg = cases_by_age_erva.groupby(['Time', 'Age'], as_index=False).sum()
age_agg.loc[age_agg['Time'] == 'Year 2021 Week 10', ]

,Time,Age,val
189,Year 2021 Week 10,0-14,490
190,Year 2021 Week 10,15-64,4253
191,Year 2021 Week 10,65+,193


In [61]:
cases_age.loc[cases_age['Time'] == 'Year 2021 Week 10', ]

,Time,age_group,val
252,Year 2021 Week 10,0-14,549
253,Year 2021 Week 10,15-64,4201
254,Year 2021 Week 10,65+,186
255,Year 2021 Week 10,All,4936


In [1]:
from fetch_data import construct_cases_age_erva_daily

In [4]:
cases_erva_age = construct_cases_age_erva_daily(logger)
cases_erva_age

,Time,erva,0-14,15-64,65+
0,2020-01-01,HYKS,0,0,0
1,2020-01-01,KYS,0,0,0
2,2020-01-01,OYS,0,0,0
3,2020-01-01,TAYS,0,0,0
4,2020-01-01,TYKS,0,0,0
...,...,...,...,...,...
4381,2021-12-31,KYS,0,0,0
4382,2021-12-31,OYS,0,0,0
4383,2021-12-31,TAYS,0,0,0
4384,2021-12-31,TYKS,0,0,0


In [9]:
foo = cases_erva_age.loc[cases_erva_age['Time'] == '2021-02-15', ]

In [10]:
foo.groupby('Time').sum()

,0-14,15-64,65+
Time,,,
2021-02-15,46,554,16


In [22]:
cases_erva_age.sort_values(['Time', 'erva'])
cases_erva_age_npy = cases_erva_age.values
cases_erva_age_npy = cases_erva_age_npy[:, -3:]
dates_ervas, ages = cases_erva_age_npy.shape
assert dates_ervas % 6 == 0
dates = int(dates_ervas/6)
cases_erva_age_npy = cases_erva_age_npy.reshape(dates, 6, ages)
cases_erva_age_npy[370, :]

array([[6, 154, 6],
       [0, 7, 2],
       [4, 29, 3],
       [0, 16, 0],
       [2, 31, 1],
       [0, 2, 0]], dtype=object)

In [20]:
dates_ervas % 6

0

In [27]:
cases_erva_age.columns[2:]

Index(['0-14', '15-64', '65+'], dtype='object')

In [17]:
from fetch_data import (
    compatment_values_daily, construct_thl_vaccines_erva_daily,
)
import os
import logging
import pandas as pd
import numpy as np

curr_dir = os.getcwd()
stats_dir = os.path.join(curr_dir, 'stats')
logger = logging.getLogger(__name__)

In [2]:
erva_pop_file = os.path.join(stats_dir, 'erva_population_age_2020.csv')

In [3]:
compart_df = compatment_values_daily(logger, erva_pop_file)
vacc_df = construct_thl_vaccines_erva_daily(logger)

In [4]:
compart_df

,date,erva,age,susceptible,infected detected,infected undetected,infected,exposed,recovered detected,recovered undetected,recovered
0,2020-01-01,HYKS,0-14,344271,0,0,0,0,0,0,0
1,2020-01-02,HYKS,0-14,344271,0,0,0,0,0,0,0
2,2020-01-03,HYKS,0-14,344271,0,0,0,0,0,0,0
3,2020-01-04,HYKS,0-14,344271,0,0,0,0,0,0,0
4,2020-01-05,HYKS,0-14,344271,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
726,2021-12-27,Åland,65+,6895,0,0,0,0,18,18,36
727,2021-12-28,Åland,65+,6895,0,0,0,0,18,18,36
728,2021-12-29,Åland,65+,6895,0,0,0,0,18,18,36
729,2021-12-30,Åland,65+,6895,0,0,0,0,18,18,36


In [13]:
vacc_df.loc[(vacc_df['date'] >= '2020-12-21') & (vacc_df['age'] == '15-64')]

,date,erva,age,First,Second
1,2020-12-21,HYKS,15-64,4.0,0.0
4,2020-12-22,HYKS,15-64,4.0,0.0
7,2020-12-23,HYKS,15-64,4.0,0.0
10,2020-12-24,HYKS,15-64,3.0,0.0
13,2020-12-25,HYKS,15-64,3.0,0.0
...,...,...,...,...,...
2485,2021-04-14,Åland,15-64,1.0,0.0
2488,2021-04-15,Åland,15-64,1.0,0.0
2491,2021-04-16,Åland,15-64,0.0,0.0
2494,2021-04-17,Åland,15-64,0.0,0.0


In [24]:
res = pd.merge(compart_df, vacc_df, on=['date', 'erva', 'age'], how='left')

,date,erva,age,susceptible,infected detected,infected undetected,infected,exposed,recovered detected,recovered undetected,recovered,First,Second
355,2020-12-21,HYKS,0-14,341885,94,94,188,190,1004,1004,2008,0.0,0.0
356,2020-12-22,HYKS,0-14,341851,92,92,184,190,1023,1023,2046,0.0,0.0
357,2020-12-23,HYKS,0-14,341837,95,95,190,172,1036,1036,2072,0.0,0.0
358,2020-12-24,HYKS,0-14,341829,95,95,190,158,1047,1047,2094,0.0,0.0
359,2020-12-25,HYKS,0-14,341825,86,86,172,154,1060,1060,2120,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13153,2021-12-27,Åland,65+,6895,0,0,0,0,18,18,36,NaN,NaN
13154,2021-12-28,Åland,65+,6895,0,0,0,0,18,18,36,NaN,NaN
13155,2021-12-29,Åland,65+,6895,0,0,0,0,18,18,36,NaN,NaN
13156,2021-12-30,Åland,65+,6895,0,0,0,0,18,18,36,NaN,NaN


In [20]:
res['Firts dose cumulative'] = res.groupby(['erva', 'age'])['First'].cumsum()
res

,date,erva,age,susceptible,infected detected,infected undetected,infected,exposed,recovered detected,recovered undetected,recovered,First,Second,Firts dose cumulative
0,2020-12-21,HYKS,0-14,341885,94,94,188,190,1004,1004,2008,0.0,0.0,0.0
1,2020-12-22,HYKS,0-14,341851,92,92,184,190,1023,1023,2046,0.0,0.0,0.0
2,2020-12-23,HYKS,0-14,341837,95,95,190,172,1036,1036,2072,0.0,0.0,0.0
3,2020-12-24,HYKS,0-14,341829,95,95,190,158,1047,1047,2094,0.0,0.0,0.0
4,2020-12-25,HYKS,0-14,341825,86,86,172,154,1060,1060,2120,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,2021-04-14,Åland,65+,6895,0,0,0,0,18,18,36,2.0,0.0,5007.0
2138,2021-04-15,Åland,65+,6895,0,0,0,0,18,18,36,2.0,0.0,5009.0
2139,2021-04-16,Åland,65+,6895,0,0,0,0,18,18,36,2.0,0.0,5011.0
2140,2021-04-17,Åland,65+,6895,0,0,0,0,18,18,36,2.0,0.0,5013.0


In [12]:
res.loc[(res['date'] >= '2020-12-21') & (res['age'] == '15-64')]

,date,erva,age,susceptible,infected detected,infected undetected,infected,exposed,recovered detected,recovered undetected,recovered,First,Second
119,2020-12-21,HYKS,15-64,1371264,1128,1128,2256,2208,17775,17775,35550,4.0,0.0
120,2020-12-22,HYKS,15-64,1371006,1099,1099,2198,2110,17982,17982,35964,4.0,0.0
121,2020-12-23,HYKS,15-64,1370728,1104,1104,2208,2022,18160,18160,36320,4.0,0.0
122,2020-12-24,HYKS,15-64,1370628,1055,1055,2110,1854,18343,18343,36686,3.0,0.0
123,2020-12-25,HYKS,15-64,1370498,1011,1011,2022,1794,18482,18482,36964,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,2021-04-14,Åland,15-64,17666,5,5,10,0,274,274,548,1.0,0.0
2019,2021-04-15,Åland,15-64,17666,4,4,8,0,275,275,550,1.0,0.0
2020,2021-04-16,Åland,15-64,17666,0,0,0,0,279,279,558,0.0,0.0
2021,2021-04-17,Åland,15-64,17666,0,0,0,0,279,279,558,0.0,0.0


In [21]:
compart_df

,date,erva,age,susceptible,infected detected,infected undetected,infected,exposed,recovered detected,recovered undetected,recovered
0,2020-01-01,HYKS,0-14,344271,0,0,0,0,0,0,0
1,2020-01-02,HYKS,0-14,344271,0,0,0,0,0,0,0
2,2020-01-03,HYKS,0-14,344271,0,0,0,0,0,0,0
3,2020-01-04,HYKS,0-14,344271,0,0,0,0,0,0,0
4,2020-01-05,HYKS,0-14,344271,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
726,2021-12-27,Åland,65+,6895,0,0,0,0,18,18,36
727,2021-12-28,Åland,65+,6895,0,0,0,0,18,18,36
728,2021-12-29,Åland,65+,6895,0,0,0,0,18,18,36
729,2021-12-30,Åland,65+,6895,0,0,0,0,18,18,36


In [22]:
vacc_df

,date,erva,age,First,Second
0,2020-12-21,HYKS,0-14,0.0,0.0
1,2020-12-21,HYKS,15-64,4.0,0.0
2,2020-12-21,HYKS,65+,0.0,0.0
3,2020-12-22,HYKS,0-14,0.0,0.0
4,2020-12-22,HYKS,15-64,4.0,0.0
...,...,...,...,...,...
2494,2021-04-17,Åland,15-64,0.0,0.0
2495,2021-04-17,Åland,65+,2.0,0.0
2496,2021-04-18,Åland,0-14,0.0,0.0
2497,2021-04-18,Åland,15-64,0.0,0.0
